<a href="https://colab.research.google.com/github/luvsaxena279/generative-ai-resume-helper/blob/main/GenAI_Resume_Helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [1]:
!pip install llama-index chromadb sentence-transformers gradio llama-index-embeddings-huggingface llama-index-llms-groq pypdf

In [2]:
import getpass, os

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ API key: ")
print("Key saved in environment!")D

Enter your GROQ API key: ··········
Key saved in environment!


In [3]:
import os

os.makedirs("data", exist_ok=True)
print(os.listdir())

['.config', 'data', 'sample_data']


In [4]:
import os, subprocess, textwrap, sys, math  # safe imports, ignore if some unused

os.makedirs("data", exist_ok=True)

!wget -O data/resume-sample.pdf "https://msnlabs.com/img/resume-sample.pdf"

print("Files in data folder:", os.listdir("data"))

--2025-12-15 14:02:39--  https://msnlabs.com/img/resume-sample.pdf
Resolving msnlabs.com (msnlabs.com)... 50.62.140.77
Connecting to msnlabs.com (msnlabs.com)|50.62.140.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301808 (295K) [application/pdf]
Saving to: ‘data/resume-sample.pdf’

data/resume-sample. 100%[===================>] 294.73K  --.-KB/s    in 0.03s   

2025-12-15 14:02:40 (10.9 MB/s) - ‘data/resume-sample.pdf’ saved [301808/301808]

Files in data folder: ['resume-sample.pdf']


In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load PDFs from the data folder
documents = SimpleDirectoryReader("data").load_data()
print("Loaded documents:", len(documents))

# Create embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build index
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
print("✅ Index created!")

Loaded documents: 13


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Index created!


In [6]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama-3.1-8b-instant")  # fast, free-tier model
query_engine = index.as_query_engine(llm=llm)

question = "List the main skills mentioned across these resume samples."
response = query_engine.query(question)
print(response)

The main skills mentioned across these resume samples include:

- Analytical skills
- Problem-solving skills
- Customer service skills
- Technical skills in operating systems (Windows, UNIX/Linux)
- Technical support skills (installation, configuration, and troubleshooting of hardware and software)
- Programming languages (Visual Basic, C, C++, Visual C++, Java, HTML, XML, ASP.NET)
- Database management skills (relational databases, SQL, PL/SQL, MS Access)
- Applications skills (MS Office, MS Project)


In [7]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama-3.1-8b-instant")  # fast, free-tier model
query_engine = index.as_query_engine(llm=llm)

question = "What kind of jobs are these resumes targeting?"
response = query_engine.query(question)
print(response)

These resumes appear to be targeting jobs in the field of Information Systems, Computer Science, or related technical fields, such as software development, technical support, or network administration.


In [8]:
import gradio as gr

def chat_fn(message, history):
    response = query_engine.query(message)
    return str(response)

demo = gr.ChatInterface(
    fn=chat_fn,
    title="Resume Helper Chatbot",
    description="Ask questions about the uploaded resume samples."
)

demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d90fb4753f80a23ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
